In [10]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

class Machine(Base):
    """The Machine Info table"""
    __tablename__ = 'machine'
    id = Column(Integer, primary_key=True)
    machine_name = Column(String(32))
    datetime = Column(DateTime, default=datetime.utcnow)
    sensors = relationship(
        'Sensor',
        collection_class=attribute_mapped_collection('name'),
        cascade='all, delete-orphan')
    sensor_values = association_proxy(
        'sensors', 'value',
        creator=lambda k, v: Sensor(name=k, value=v))

class Sensor(Base):
    """The Sensor table"""
    __tablename__ = 'sensor'
    machine_id = Column(Integer, ForeignKey('machine.id'),
                           primary_key=True)
    # Note that this could be a numeric ID as well
    name = Column(String(16), primary_key=True)
    value = Column(String(32))

In [28]:
engine = create_engine('sqlite:///test_db.db', echo=True)

In [29]:
Base.metadata.create_all(engine)

2020-11-29 19:35:26,931 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-29 19:35:26,933 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 19:35:26,936 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-29 19:35:26,937 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 19:35:26,938 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("machine")
2020-11-29 19:35:26,940 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 19:35:26,942 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("sensor")
2020-11-29 19:35:26,943 INFO sqlalchemy.engine.base.Engine ()


In [30]:
m = Machine(machine_name='Steam Machine')

In [31]:
m.sensor_values['sensor1'] = 'some interesting value'

In [32]:
session = Session(engine)
session.add(m)

In [33]:
session.commit()

2020-11-29 19:35:36,550 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-29 19:35:36,553 INFO sqlalchemy.engine.base.Engine INSERT INTO machine (machine_name, datetime) VALUES (?, ?)
2020-11-29 19:35:36,555 INFO sqlalchemy.engine.base.Engine ('Steam Machine', '2020-11-29 18:35:36.553425')
2020-11-29 19:35:36,559 INFO sqlalchemy.engine.base.Engine INSERT INTO sensor (machine_id, name, value) VALUES (?, ?, ?)
2020-11-29 19:35:36,560 INFO sqlalchemy.engine.base.Engine (2, 'sensor1', 'some interesting value')
2020-11-29 19:35:36,562 INFO sqlalchemy.engine.base.Engine COMMIT


In [34]:
session.close()